<a href="https://colab.research.google.com/github/papercore-dev/perfectly-jogyo/blob/main/vc-run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **⚠ GPU나 TPU를 이용한 비트코인 및 가상화폐 채굴은 금지되어 있습니다. ⚠**

# 🐸TTS × YourTTS
## VITS 기반 TTS 엔진 제작기

제가 장담하건대, 저 포함 여러분들의 99% 이상은 TTS가 궁금하지, 그것을 운용시키는 무슨 Pytorch라던지, GAN, KSS, LJSpeech 같은 건 들어보지도 않았을 뿐더러 알기도 싫으실 겁니다.

그래서 간단히 이 사이트 사용법에 대해서 설명을 하자면, TTS처럼 글자 입력해서 목소리가 나오는 건 절대 아닙니다. (gTTS를 쓰면 되긴 하지만, 귀찮아서 하진 않습니다.)

따라서 여러분의 아무 말이나 하는 WAV 형식의 목소리 파일 (50건 이상 권장)과 말하고자 하는 문장이 포함된 WAV 형식의 목소리 파일 1건을 가져오셔서 진행하시면 되겠습니다.

아래의 방법을 따라주세요!

**✨ 추천 링크:**
* 유튜브 영상에서 목소리를 추출해서 TTS를 제작하는 Colab [->](https://colab.research.google.com/github/papercore-dev/perfectly-jogyo/blob/main/transcribe.ipynb)
* 여러가지 보이스들 [->](https://github.com/papercore-dev/perfectly-jogyo/tree/main/models)

##🚀 **TTS 모델 설정하기**
*숨겨진 셀 10개* 라고 적힌 부분 왼쪽의 큼지막한 재생 버튼을 눌러서 실행하시면 됩니다.

보안 관련 경고가 나타나면은 **무시하고 계속**을 누르시면 됩니다.

### 🐸TTS 설정하기
Coqui라는 회사에서 제작한 TTS인데요, Coqui는 음성 인식 및 음성 합성을 주 업무로 하고 있는 회사에요.


In [ ]:
!git clone https://github.com/Edresson/Coqui-TTS -b multilingual-torchaudio-SE TTS
!pip install -q -e TTS/
!pip install -q torchaudio==0.9.0

Cloning into 'TTS'...
remote: Enumerating objects: 22476, done.
remote: Counting objects: 100% (1658/1658), done.
remote: Compressing objects: 100% (549/549), done.
remote: Total 22476 (delta 1152), reused 1439 (delta 1070), pack-reused 20818
Receiving objects: 100% (22476/22476), 133.45 MiB | 29.96 MiB/s, done.
Resolving deltas: 100% (16261/16261), done.


###✨ TTS 사전 설정값 다운로드
사전 설정값, 즉 체크포인트 없이는 GPU 사용량이 높은 트레이닝 과정을 거쳐야 하는데, 이거 하시는 분들은 그런거 잘 모르실 거잖아요? 그래서 미리 해봤습니다 :)

In [ ]:
# 사전 설정값 (체크포인트)을 구글 드라이브에서 받습니다.

# download config  
!gdown --id 1-PfXD66l1ZpsZmJiC-vhL055CDSugLyP
# download language json 
! gdown --id 1_Vb2_XHqcC0OcvRF82F883MTxfTRmerg
# download speakers json
! gdown --id 1SZ9GE0CBM-xGstiXH2-O2QWdmSXsBKdC -O speakers.json
# download checkpoint
! gdown --id 1sgEjHt0lbPSEw9-FSbC_mBoOPwNi87YR -O best_model.pth.tar

### ✨ 필요한 라이브러리 가져오기

In [ ]:
import sys
TTS_PATH = "TTS/"

# add libraries into environment
sys.path.append(TTS_PATH) # set this if TTS is not installed globally

import os
import string
import time
import argparse
import json

import numpy as np
import IPython
from IPython.display import Audio


import torch

from TTS.tts.utils.synthesis import synthesis
from TTS.tts.utils.text.symbols import make_symbols, phonemes, symbols
try:
  from TTS.utils.audio import AudioProcessor
except:
  from TTS.utils.audio import AudioProcessor


from TTS.tts.models import setup_model
from TTS.config import load_config
from TTS.tts.models.vits import *

### 경로 설정

In [ ]:
OUT_PATH = 'out/'

# create output path
os.makedirs(OUT_PATH, exist_ok=True)

# model vars 
MODEL_PATH = 'best_model.pth.tar'
CONFIG_PATH = 'config.json'
TTS_LANGUAGES = "language_ids.json"
TTS_SPEAKERS = "speakers.json"
USE_CUDA = torch.cuda.is_available()

### 모델 복원

In [ ]:
# load the config
C = load_config(CONFIG_PATH)


# load the audio processor
ap = AudioProcessor(**C.audio)

speaker_embedding = None

C.model_args['d_vector_file'] = TTS_SPEAKERS
C.model_args['use_speaker_encoder_as_loss'] = False

model = setup_model(C)
model.language_manager.set_language_ids_from_file(TTS_LANGUAGES)
# print(model.language_manager.num_languages, model.embedded_language_dim)
# print(model.emb_l)
cp = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
# remove speaker encoder
model_weights = cp['model'].copy()
for key in list(model_weights.keys()):
  if "speaker_encoder" in key:
    del model_weights[key]

model.load_state_dict(model_weights)


model.eval()

if USE_CUDA:
    model = model.cuda()

# synthesize voice
use_griffin_lim = False

##🚀 **음성 인코더 설정**
음성 합성은 다 끝났는데, 결과물이 없다면 반쪽짜리 TTS가 되는지라, 인코더도 설정하려 해요.

아까처럼 ▶️ 버튼을 눌러주세요!

### ✨ 라이브러리 설치
음성의 질을 균등하게 해주는 FFMPEG Normalize와, 여러 음성을 합치는 Pydub을 설치해요.

In [6]:
! pip install -q pydub ffmpeg-normalize

### ✨ 경로 설정
체크포인트 몇개만 더 받을게요!

In [7]:
CONFIG_SE_PATH = "config_se.json"
CHECKPOINT_SE_PATH = "SE_checkpoint.pth.tar"

# download config 
! gdown --id  19cDrhZZ0PfKf2Zhr_ebB-QASRw844Tn1 -O $CONFIG_SE_PATH
# download checkpoint  
! gdown --id   17JsW6h6TIh7-LkU2EvB_gnNrPcdBxt7X -O $CHECKPOINT_SE_PATH

Downloading...
From: https://drive.google.com/uc?id=19cDrhZZ0PfKf2Zhr_ebB-QASRw844Tn1
To: /content/config_se.json
100% 3.49k/3.49k [00:00<00:00, 3.09MB/s]
Downloading...
From: https://drive.google.com/uc?id=17JsW6h6TIh7-LkU2EvB_gnNrPcdBxt7X
To: /content/SE_checkpoint.pth.tar
100% 44.6M/44.6M [00:00<00:00, 58.2MB/s]


###Imports

In [8]:
from TTS.tts.utils.speakers import SpeakerManager
from pydub import AudioSegment
from google.colab import files
import librosa

###Load the Speaker encoder

In [9]:
SE_speaker_manager = SpeakerManager(encoder_model_path=CHECKPOINT_SE_PATH, encoder_config_path=CONFIG_SE_PATH, use_cuda=USE_CUDA)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:512
 | > power:1.5
 | > preemphasis:0.97
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:False
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:False
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > stats_path:None
 | > base:10
 | > hop_length:160
 | > win_length:400


###Define helper function

In [10]:
def compute_spec(ref_file):
  y, sr = librosa.load(ref_file, sr=ap.sample_rate)
  spec = ap.spectrogram(y)
  spec = torch.FloatTensor(spec).unsqueeze(0)
  return spec

##🚀 **음성 합성하기**
자~ 이제 설정이 끝났습니다!
이제부터 음성 합성을 하고자 하는데요,

이름과 달리 그리 어렵지 않습니다.
한번 직접 해봐요!

###✨음성 업로드

In [ ]:
#@title 🎤 **새 보이스 제작하기**<br>원하는 보이스가 없다면, 새로운 보이스를 제작해도 좋아요.<br>**보이스 파일의 형식은 WAV (파형 오디오) 형식의 파일 필수, 50건 이상의 파일 권장 (좋은 결과를 얻을 수 있어요)입니다.**

print("변조하고자 하는 이의 원본 목소리를 업로드합니다.:")
target_files = files.upload()
target_files = list(target_files.keys())
for sample in target_files:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f

In [ ]:
#@title 🎤 **아무 말이나 올리기**<br>자신의 보이스를 올리세요.<br>**보이스 파일의 형식은 WAV (파형 오디오) 형식의 파일 필수, 50건 이상의 파일 권장 (좋은 결과를 얻을 수 있어요)입니다.**
print("Select driving speaker reference audios files:")
driving_files = files.upload()
driving_files = list(driving_files.keys())
for sample in driving_files:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f

In [ ]:
#@title 🎤 **대상이 할 말을 올리기**<br>**보이스 파일의 형식은 WAV (파형 오디오) 형식의 파일 필수, 1건의 파일 필수입니다.**

print("변조하고 싶은 말을 올리세요.")
driving_file = files.upload()
driving_file = list(driving_file.keys())
for sample in driving_file:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f
driving_file = driving_file[0]

###✨ AI의 학습 단계

In [ ]:
#@title 🎤 **새 보이스 학습하기**<br>보이스 학습이 완료되었다면, 이 값을 복사해서, <a href="https://github.com/papercore-dev/perfectly-jogyo/issues/new">깃허브 이슈를 제작</a>하여 누구나 사용할 수 있게 도와주시면 감사하겠습니다.

target_emb = SE_speaker_manager.compute_d_vector_from_clip(target_files)
target_emb = torch.FloatTensor(target_emb).unsqueeze(0)
print('보이스 값입니다. 다음 줄부터 복사해주세요! \n')
print(target_emb)

In [ ]:
#@title 🎤 **[테스트용] Driving 보이스 학습하기**<br>일반 사용자 분들은 실행하지 않는 것을 권장합니다!

driving_emb = SE_speaker_manager.compute_d_vector_from_clip(driving_files)
driving_emb = torch.FloatTensor(driving_emb).unsqueeze(0)
print('보이스 값입니다. 다음 줄부터 복사해주세요! \n')
print(driving_emb)

In [ ]:
#@title 🎤 **내 보이스 학습하고 합성하기**

target_emb = eval(input ("보이스 txt파일의 값을 입력하세요 :"))
driving_emb = torch.tensor([[ 2.0607e-02, -4.7483e-02, -1.5167e-02,  6.3849e-02, -7.2371e-03,
          1.1932e-02, -2.0498e-02, -4.2601e-02, -2.1796e-04,  3.3028e-02,
         -1.4663e-02,  1.0862e-02, -3.5605e-02,  2.2984e-03,  1.7091e-02,
          1.1683e-02,  5.8630e-02,  8.0149e-02, -9.2952e-03,  2.4638e-02,
         -5.0337e-02,  5.9797e-03,  1.6997e-02, -3.0498e-02, -1.5955e-02,
         -2.8325e-02,  1.5620e-02, -9.4198e-03,  8.1197e-04,  1.6572e-02,
         -1.1781e-02, -1.6700e-02,  3.4180e-02,  3.2427e-02,  8.9045e-03,
          2.6123e-02, -4.3297e-02, -4.2151e-02, -8.5348e-03, -1.0142e-02,
         -1.1794e-02,  3.7384e-02, -2.8211e-02,  2.0001e-02, -1.7376e-02,
          9.2483e-02, -2.6439e-02, -4.8970e-02, -1.8065e-02, -2.7430e-02,
          1.9960e-02,  4.8954e-02,  1.9096e-02, -2.7000e-02, -3.1905e-02,
         -1.3921e-02,  1.6273e-02, -3.3391e-02,  2.3612e-02, -1.4117e-02,
         -2.1112e-02, -8.0526e-03, -1.0543e-01, -1.9143e-02, -1.2505e-02,
          1.7720e-02, -3.3286e-02,  1.1507e-02, -4.1603e-02,  7.2590e-02,
         -2.8703e-02, -1.1724e-02,  1.7038e-02,  1.4717e-02, -4.2994e-03,
         -5.4472e-04,  1.4546e-02, -1.0977e-02,  5.6513e-03,  1.1174e-02,
         -1.6389e-02, -2.1208e-02,  7.3852e-04,  2.4175e-02, -4.1490e-02,
          1.0472e-02, -1.6937e-02,  7.4035e-03, -2.3294e-02,  5.7173e-02,
         -1.1106e-02,  4.3161e-02,  5.8434e-02,  3.5507e-02,  4.6284e-02,
         -5.1607e-03,  2.5490e-02, -2.0014e-02, -3.3969e-02,  2.2961e-03,
         -3.5547e-04,  1.5166e-02, -3.2927e-02, -3.8913e-02,  3.3861e-04,
         -3.7059e-03,  1.2104e-02,  7.6911e-03,  1.9654e-02,  2.7509e-02,
          4.7102e-02, -8.8850e-03,  5.5404e-03,  7.2717e-02, -3.1020e-03,
          7.3845e-03, -6.2775e-02,  8.4483e-03, -1.8361e-02, -8.9340e-04,
         -7.3868e-02,  1.8606e-02,  4.4027e-03,  3.6980e-02,  1.0102e-02,
          2.9487e-02, -1.1321e-02, -2.1859e-02, -5.5144e-03,  3.1786e-02,
          2.5820e-03, -2.0113e-02, -3.7759e-03,  4.9461e-03,  3.2954e-02,
         -1.7715e-02,  2.3900e-02,  3.3668e-02,  1.8261e-02, -4.8252e-02,
         -5.7878e-03, -3.7407e-02, -5.6142e-03, -2.6982e-02, -9.7218e-03,
         -3.5233e-02, -3.6180e-02, -4.3366e-02,  3.4726e-03,  1.6853e-02,
         -1.2368e-02, -1.2921e-02,  1.5221e-02,  6.9922e-03, -3.0671e-02,
         -2.5996e-02,  7.1259e-03, -4.6916e-02,  1.0785e-02,  6.0995e-03,
          4.9667e-04, -1.6177e-02, -1.4517e-02, -1.7225e-02, -1.2194e-02,
          3.7305e-03, -2.1514e-02,  1.6469e-03,  1.6413e-02, -5.3201e-04,
          9.0034e-03,  1.2636e-02,  1.3496e-02, -6.4854e-03,  2.6079e-03,
          2.3148e-02, -9.0852e-03, -3.8940e-02,  1.1323e-02,  3.6199e-02,
         -2.7328e-02,  2.5615e-02, -4.3465e-02,  1.7853e-02, -1.4671e-02,
          1.9717e-02,  6.3875e-02, -4.8026e-02, -2.7620e-02, -3.7121e-02,
          4.2220e-02, -1.0884e-02,  7.9302e-03,  9.1921e-04,  5.2248e-03,
         -2.1019e-02, -1.3801e-02,  1.4611e-02, -1.9237e-02, -1.7790e-02,
         -1.8399e-05,  4.4806e-02,  1.3406e-02, -8.4835e-03,  2.8949e-02,
         -5.2899e-03, -3.6356e-04, -1.4165e-02,  1.0454e-02, -2.5980e-02,
         -3.7522e-02, -1.7267e-02,  9.5179e-02, -1.8892e-03,  2.5673e-02,
         -5.7823e-03, -3.6506e-02,  1.0208e-02, -1.7737e-02,  2.7244e-03,
         -1.9583e-02, -6.3506e-03,  5.3978e-03,  2.0272e-03,  3.2569e-02,
         -1.7702e-02, -2.6163e-02, -3.2982e-02,  3.0730e-02,  6.1114e-02,
         -3.6887e-03, -1.8082e-02, -3.1809e-03,  2.7065e-02,  1.5305e-02,
         -1.0506e-02,  3.8729e-05,  9.9690e-03,  2.6982e-02, -1.5314e-02,
         -3.6345e-02, -8.0890e-04,  9.9015e-03, -1.1876e-02,  1.5658e-02,
          7.7905e-02, -4.2189e-04,  1.3048e-02, -6.0389e-03, -6.5637e-03,
          1.0985e-03,  1.0680e-03, -3.1170e-02,  3.2110e-03, -1.2219e-02,
         -2.6645e-02, -4.8939e-04,  9.3718e-03,  3.0890e-02,  3.7995e-02,
          2.5451e-02,  6.8318e-04, -8.4691e-03,  8.4007e-03, -1.0698e-02,
          3.8441e-03, -3.3538e-02, -2.2682e-02, -1.6726e-02, -1.0376e-02,
         -6.2090e-02, -6.1512e-03, -4.7538e-02, -4.6577e-02,  2.0555e-02,
          3.4271e-03,  2.9118e-02, -1.5088e-02, -2.7277e-02, -2.2791e-02,
          4.9562e-04,  5.3264e-03,  4.3749e-02, -2.9660e-02, -1.2207e-02,
          3.3014e-03, -1.8272e-02,  2.6711e-02, -1.5238e-04, -8.5070e-03,
          4.7260e-03,  2.0151e-02, -1.1312e-02, -1.3684e-01, -2.4037e-02,
          1.2107e-02,  2.0118e-02,  1.4506e-02,  1.5670e-02,  1.2651e-02,
          9.4940e-02,  3.1900e-02, -9.1165e-03,  1.5698e-01, -1.1373e-02,
          9.2604e-03,  4.2594e-03, -2.2972e-02,  3.4227e-03, -3.1953e-02,
          6.8770e-03,  2.3766e-02, -1.4643e-02,  3.6150e-02,  2.0819e-01,
          6.5621e-04,  3.5131e-03, -3.1327e-03,  2.7826e-05, -2.3118e-02,
          5.5539e-03, -1.3500e-02, -4.9612e-02,  2.2256e-02, -6.2585e-03,
         -2.8275e-02,  1.4489e-02, -2.8864e-02, -2.1792e-02,  2.4062e-02,
         -2.1187e-02, -2.1493e-02, -2.0939e-02, -2.2362e-02, -1.1333e-02,
         -1.7641e-02,  1.0395e-02,  5.1185e-02, -3.0456e-02, -1.8137e-02,
          7.4908e-03,  1.5546e-02, -1.1329e-02, -2.8293e-03, -3.0929e-02,
          7.0752e-03,  1.7143e-02, -1.9176e-03,  1.3922e-03,  6.8630e-03,
         -5.2138e-02, -3.6983e-03,  9.3217e-04, -2.0889e-02,  3.4480e-02,
         -7.3457e-04, -5.0357e-02, -1.2712e-04, -3.8634e-02,  8.1964e-04,
          1.0537e-02, -1.4996e-02, -1.6492e-02, -1.5458e-02,  1.0356e-02,
         -2.1878e-02, -5.0020e-02, -6.0605e-02, -1.0515e-02, -3.1447e-02,
         -2.6701e-02, -5.0244e-02,  7.9348e-03,  2.3307e-02,  2.2586e-02,
         -1.0340e-02, -1.3252e-02,  9.3583e-03, -6.1286e-04,  1.1038e-02,
         -9.1719e-03,  1.3126e-02, -2.7325e-02,  2.0955e-03,  2.6832e-03,
         -9.8714e-03, -4.8121e-03,  5.8122e-03,  5.3213e-03, -8.8641e-03,
          2.5424e-02,  1.8870e-02,  3.3907e-03,  4.7266e-03, -2.1751e-02,
          2.0781e-02,  6.5494e-02,  3.6118e-02, -6.6627e-02, -6.5553e-04,
         -3.0037e-02, -1.5212e-02, -2.5270e-02,  2.3586e-02, -7.5385e-03,
         -5.0600e-03, -2.6678e-02, -1.3220e-02,  6.0755e-03, -1.0803e-02,
         -4.2818e-02,  6.5829e-03,  7.8916e-03,  3.0939e-03, -6.6919e-02,
         -2.1552e-02,  4.3844e-02,  8.8731e-03, -8.9896e-03,  2.4915e-02,
          5.8537e-02, -8.0762e-04, -6.5824e-02,  2.1985e-02, -4.0750e-02,
          1.8957e-02, -6.6045e-03, -8.3403e-03,  4.3992e-03, -9.4134e-03,
          2.3777e-05,  2.8043e-02,  9.1162e-02,  5.1641e-03,  1.6267e-02,
         -1.7890e-02,  8.4119e-03,  3.1839e-01, -1.4389e-02,  1.2436e-02,
         -8.1528e-03,  1.8799e-02,  1.9135e-02, -9.4485e-04,  9.3106e-03,
         -2.2558e-03, -1.5250e-03,  1.0971e-02,  1.3307e-02, -8.2865e-02,
         -1.7207e-02, -2.5538e-02,  4.9789e-03, -1.8967e-02, -9.5155e-03,
         -1.0029e-02, -2.7638e-02,  3.0208e-02,  7.5988e-03,  1.5176e-02,
          1.8921e-02, -2.5433e-02, -5.0332e-02, -1.0810e-02, -4.1474e-03,
         -4.4818e-02,  2.3211e-02, -5.5956e-02,  1.6723e-02,  1.1907e-03,
         -4.3461e-03, -5.1734e-02, -4.7161e-02,  5.2489e-04, -1.3265e-02,
         -1.3532e-04, -3.2849e-02, -8.1054e-03,  1.9447e-02,  9.9412e-03,
         -4.5688e-03,  8.6408e-03, -6.9994e-02, -3.3567e-02, -6.4054e-03,
          5.3188e-03, -3.7919e-02, -1.9843e-02, -2.1694e-02, -4.7311e-02,
          2.3532e-02,  7.4324e-03, -2.1364e-02, -9.8856e-03,  1.0611e-02,
         -1.1305e-02, -1.5942e-02,  2.7240e-02,  1.5778e-02,  2.2057e-02,
          2.7114e-03,  1.9219e-02, -1.0474e-02,  1.0824e-02, -3.3887e-03,
         -1.8861e-03, -1.1451e-02, -2.9568e-02,  2.3484e-03, -3.4740e-02,
         -8.7603e-03,  1.9606e-02]])
if driving_emb is None:
    driving_emb = SE_speaker_manager.compute_d_vector_from_clip(driving_files)
    driving_emb = torch.FloatTensor(driving_emb).unsqueeze(0)

driving_spec = compute_spec(driving_file)
y_lengths = torch.tensor([driving_spec.size(-1)])
if USE_CUDA:
    ref_wav_voc, _, _ = model.voice_conversion(driving_spec.cuda(), y_lengths.cuda(), driving_emb.cuda(), target_emb.cuda())
    ref_wav_voc = ref_wav_voc.squeeze().cpu().detach().numpy()
else:
    ref_wav_voc, _, _ = model.voice_conversion(driving_spec, y_lengths, driving_emb, target_emb)
    ref_wav_voc = ref_wav_voc.squeeze().detach().numpy()


print("결과물입니다:")
IPython.display.display(Audio(ref_wav_voc, rate=ap.sample_rate))